# **Run first before everything**

In [1]:
# top-refs-from-paper: retrieve top citations from a scientific paper using Crossref
# Copyright (C) 2023 Tomasz Bednarek

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU Affero General Public License as published
# by the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU Affero General Public License for more details.

# You should have received a copy of the GNU Affero General Public License
# along with this program.  If not, see <https://www.gnu.org/licenses/>.

from collections import Counter
from time import perf_counter, sleep
import argparse, json, os, pip, re

before = perf_counter()

def install(package):
  pip.main(['install', package])

def install_all_packages(modules_to_try):
  os.system('python -m pip install --upgrade pip > $null')
  for module in modules_to_try:
    try:
      __import__(module)        
    except ImportError as e:
      install(e.name)

install_all_packages(['bs4', 'prettytable', 'requests'])
import requests
from bs4 import BeautifulSoup
from prettytable import PrettyTable

flag = True
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

def get_references(doi_link):
  response = None

  try:
    flag = True
    response = requests.get(f'https://api.crossref.org/works/{doi_link}')
  except TimeoutError:
    print(f'Timeout for {doi_link}')
    sleep(60)
    if flag: 
      flag = False
      return get_references(doi_link)
    else: 
      print('Critical error: shutting down the script!')
      exit()

  output = None

  try: 
    output = response.json().get('message', {}).get('reference')
  except json.decoder.JSONDecodeError:
    print(f"There is a problem with JSON output from DOI: {doi_link}")
    pass

  if output != None:
    ref_list = [ el.get('DOI') for el in output if el.get('DOI') ]
    return ref_list
  else: pass

def get_title(doi_link):
  response = None
  output = None
  
  try:
    flag = True
    response = requests.get(f'https://api.crossref.org/works/{doi_link}')
  except TimeoutError:
    print(f'Timeout for {doi_link}')
    sleep(60)
    if flag: 
      flag = False
      return get_references(doi_link)
    else: 
      print('Critical error: shutting down the script!')
      exit()

  try: 
    output = cleanhtml(response.json().get('message', {}).get('title')[0])
  except json.decoder.JSONDecodeError:
    print(f"There is a problem with title retrival from DOI: {doi_link}")
    pass

  return output

def get_ranking(temp_set, depth, length, previous_list=[]):
  next_list = []
  for link in temp_set:
    temp = get_references(link)
    if temp: next_list.extend(temp)
    else: print(f'References not found for {link}')

  next_set = set(next_list)
  previous_set = set(previous_list)
  next_set -= previous_set
  next_list.extend(previous_list)

  if depth > 1:
    get_ranking(next_set, depth - 1, length, next_list)
  else:
    res = Counter(next_list).most_common()
    my_table = PrettyTable()
    my_table.field_names = ["Count", "DOI", "Title"]

    for doi, count in res[:length]:
      title = get_title(doi)
      if title: my_table.add_row([count, doi, title])

    print('\n')
    print(my_table)
    pass  

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# **Example run for the DOI: https://doi.org/10.1016/j.susc.2005.05.030**

In [2]:
get_ranking(['https://doi.org/10.1016/j.susc.2005.05.030'], depth=3, length=20)

References not found for 10.1007/s003390051306
References not found for 10.1039/b002630l
References not found for 10.1021/la00059a027
References not found for 10.1119/1.1974502
References not found for 10.1007/978-3-642-86105-5
References not found for 10.1007/BF01399758
References not found for 10.1017/CBO9780511622564
References not found for 10.1007/s003390100674
References not found for 10.1016/0079-6816(95)00050-X
References not found for 10.1147/rd.141.0061
References not found for 10.1142/S0217979295000562
References not found for 10.1038/328299a0
References not found for 10.1016/0003-4916(76)90038-5
References not found for 10.1007/s003400050674
References not found for 10.1139/p89-178
References not found for 10.1007/978-1-4757-5107-9
References not found for 10.1103/PhysRevB.52.R14380
References not found for 10.1142/S0218625X96002539
References not found for 10.1142/S0218625X95000121
References not found for 10.1116/1.579114
References not found for 10.1103/PhysRevLett.2.393